In [1]:
import os

In [2]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [8]:
import time
import tensorflow as tf

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

    @property
    def _create_lr_scheduler(self):
        def scheduler(epoch, lr):
            if epoch % 10 == 0 and epoch > 0:
                lr = lr / 2
            return lr

        return tf.keras.callbacks.LearningRateScheduler(scheduler)

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping,
            self._create_lr_scheduler
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-07-22 10:09:02,562: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-07-22 10:09:02,574: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-22 10:09:02,576: INFO: utils: created directory at: artifacts]
[2023-07-22 10:09:02,579: INFO: utils: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-22 10:09:02,582: INFO: utils: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


AttributeError: 'WindowsPath' object has no attribute 'endswith'

In [11]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-07-22-10-16-32'